In [2]:
# test if pytorch can run in gpu
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

cuda_available = torch.cuda.is_available()
print(f"CUDA available: {cuda_available}")
if cuda_available:
    print(f"CUDA device count: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"CUDA device {i}: {torch.cuda.get_device_name(i)}")

print(torch.__version__)
print(torch.version.cuda)

CUDA available: True
CUDA device count: 1
CUDA device 0: NVIDIA GeForce RTX 4080
cuda
2.4.1+cu118
11.8


### Understand data

In [6]:
from PIL import Image
import numpy as np

image = Image.open('data/train/54.jpg')
pic = np.array(image)

print(pic.shape)

(224, 224, 3)


In [29]:
# read metadata
import pandas as pd
metadata = pd.read_csv('data/train.csv')

# check the values in stable_height
print(metadata['stable_height'].value_counts())

stable_height
2    1920
1    1920
3    1536
4    1152
5     768
6     384
Name: count, dtype: int64


### Test model accuracy

In [2]:
import torch
from torch.utils.data import DataLoader
from backbones.resnet_SE import CNN
import dataset
import os

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# Load the test dataset
def load_test_data(csv_file, image_dir, batch_size=64):
    test_dataset = dataset.StackDataset(csv_file=csv_file, image_dir=image_dir, img_size=(224), stable_height = 'stable_height', train=False, testMode=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    return test_loader

In [5]:
# Function to calculate accuracy for classification model output
def calculate_accuracy(model, test_loader, device):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  # Disable gradient calculation for evaluation
        for inputs, targets in test_loader:
            # Move data to device
            inputs, targets = inputs.to(device), targets.to(device)
            targets = targets.long()  # Ensure targets are long for classification
            
            # Get model predictions (outputs are logits)
            outputs = model(inputs)
            
            # Get the index of the max logit (predicted class for each sample)
            _, predicted = torch.max(outputs, 1)  # outputs shape: [batch_size, n_classes]
            
            # Update accuracy metrics
            total += targets.size(0)
            correct += (predicted == targets).sum().item()  # Compare predicted and actual targets
            
    if total == 0:  # Prevent division by zero
        return 0
    accuracy = 100 * correct / total
    return accuracy

In [6]:
# Function to test model accuracy
def test_model_accuracy(model_path, csv_file, image_dir):
    # Load the model
    model = CNN()  # Initialize model
    model.load_state_dict(torch.load(model_path, weights_only=True))  # Load saved weights
    model.to(device)  # Move model to the device

    # Load the test data
    test_loader = load_test_data(csv_file, image_dir)

    # print_top_5_predictions(model, test_loader)
    
    # Calculate accuracy
    accuracy = calculate_accuracy(model, test_loader, device)
    print(f"Model Accuracy on Test Set: {accuracy:.2f}%")
    return accuracy

In [10]:
csv_file='data/sample-solution.csv'
image_dir='data/test'
model_path_root='models/exp5'

import re
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text for text in re.split(r'(\d+)', s)]

# loop through all models and print accuracy
for model_file in sorted(os.listdir(model_path_root), key=natural_sort_key):
    model_path = os.path.join(model_path_root, model_file)
    print(f"Testing model: {model_path}")
    test_model_accuracy(model_path, csv_file, image_dir)


Testing model: models/exp5\model_epoch_1.pth
Model Accuracy on Test Set: 6.67%
Testing model: models/exp5\model_epoch_2.pth
Model Accuracy on Test Set: 11.93%
Testing model: models/exp5\model_epoch_3.pth
Model Accuracy on Test Set: 15.94%
Testing model: models/exp5\model_epoch_4.pth
Model Accuracy on Test Set: 3.80%
Testing model: models/exp5\model_epoch_5.pth
Model Accuracy on Test Set: 10.99%
Testing model: models/exp5\model_epoch_6.pth
Model Accuracy on Test Set: 6.98%
Testing model: models/exp5\model_epoch_7.pth
Model Accuracy on Test Set: 9.17%
Testing model: models/exp5\model_epoch_8.pth
Model Accuracy on Test Set: 7.60%
Testing model: models/exp5\model_epoch_9.pth
Model Accuracy on Test Set: 8.54%
Testing model: models/exp5\model_epoch_10.pth
Model Accuracy on Test Set: 8.59%
Testing model: models/exp5\model_epoch_11.pth
Model Accuracy on Test Set: 8.91%
Testing model: models/exp5\model_epoch_12.pth
Model Accuracy on Test Set: 9.01%
Testing model: models/exp5\model_epoch_13.pth
